In [1]:
import numpy as np
import pandas as pd

In [3]:
pd.__version__

'1.1.3'

## 目录


**0. 连接的基本概念**


**一、关系型连接**

* 1.1. 值连接
* 1.2. 索引连接


**二、方向连接**

* 2.1. concat
* 2.2. 序列与表的合并



**三、类连接操作**

* 3.1. 比较
* 3.2. 组合


**四、练习**

* Ex1：美国疫情数据集
* Ex2：实现join函数



## 正式学习内容



### 0、连接的基本概念

把两张相关的表连接在一起是一种常见的操作，这部分的理解结合**数据库SQL**会很好理解。

连接的方式有两种：

   * 一种是**关系型连接**，关系型连接我们可以理解为，表格通过*表格中的某列的值*联系在一起，同时连接的形式有 左连接left join，右连接right join，内连接（取交集）inner join，外连接（取并集）outer join

   * 一种是**方向型连接**，方向型的拼接就是不关心表之间的关系（当然表的shape要一致才行），直接**横向拼接** 或者**纵向拼接**在一起。


### 一、关系型连接


关系型连接需要注意的有两个点：

* 一个是**连接的键**，在函数里，参数往往是 on，表示以哪个关系连接在一起。

* 一个是**连接的形式**，函数参数一般是 how，表示那种连接方式，可选的一般为左连接，右连接，内连接和外连接。

另外，需要说明一下连接形式：

* **左（右）连接：** 以左（右）表键值为基准,左（右）表不存在的值则用空值表示。

* **内连接** 左右表的交集，只保留左右表都存在的值。

* **外连接** 左右表的并集，不存在的值用空值填充

具体的连接形式可以通过例子学习。


#### 1.1. 值连接


pandas中用来实现关系型值连接的函数是 **.merge**

* **df.merge**(   right,how='inner',on=None,left_on=None,right_on=None,left_index=False,right_index=False,sort=False,suffixes=('_x','_y'),copy=True,indicator=False,validate=None,)  
      
    
    参数说明


|默认参数 | 使用说明|  
|---|---|  
|right|右边的DataFrame或者Series |
|how='inner'|连接方式，默认 inner，其他可选{'left', 'right', 'outer'}|
|on=None|连接时选择的键值，默认None | 
|left_on=None|左表用于连接的键值列名 | 
|right_on=None|右表用于连接的键值列名  |
|left_index=False|左表用于连接的键值索引名 |  
|right_index=False|右表用于连接的键值索引名，（*这里和on的使用区别是什么?*） |
|sort=False|合并的表格是否按照键值排序 |
|suffixes=('_x', '_y') |两表重复列的命名方式，默认col_x，col_y | 
|copy=True|  |
|indicator=False|  |
|validate=None| |  
    


In [18]:
# 
df1 = pd.DataFrame(data={'Name':['San Zhang','Si Li','Wu Wang'],
                       'Age':['23','29','0'],
                        'Weight':['45','70','8']})

df1

,Name,Age,Weight
0,San Zhang,23,45
1,Si Li,29,70
2,Wu Wang,0,8


In [19]:
# df2
df2 = pd.DataFrame(data={'Name':['San Zhang','Si Li','Liu Zhao'],
                       'Gender':['F','M','F']})

df2

,Name,Gender
0,San Zhang,F
1,Si Li,M
2,Liu Zhao,F


In [20]:
# 参数on
df1.merge(df2,on='Name')

,Name,Age,Weight,Gender
0,San Zhang,23,45,F
1,Si Li,29,70,M


In [21]:
# 参数how
df1.merge(df2,on='Name',how='left')

,Name,Age,Weight,Gender
0,San Zhang,23,45,F
1,Si Li,29,70,M
2,Wu Wang,0,8,NaN


In [22]:
# left_on right_on 左右边键值名称不一样 可以分别指定

df1.rename(lambda x: 'df1_name' if x =='Name' else x,
           axis=1,
          inplace=True)
df1

,df1_name,Age,Weight
0,San Zhang,23,45
1,Si Li,29,70
2,Wu Wang,0,8


In [29]:
# left_on right_on 左右边键值名称不一样 可以分别指定
df1.merge(df2,
          left_on='df1_name',
          right_on = 'Name',
          how='outer')

,df1_name,Age,Weight,Name,Gender
0,San Zhang,23,45,San Zhang,F
1,Si Li,29,70,Si Li,M
2,Wu Wang,0,8,NaN,NaN
3,NaN,NaN,NaN,Liu Zhao,F


In [40]:
# left_index right_index 按行索引进行合并
# 不太好用，行索引不好处理

df1 = df1.append(pd.Series(['Qi Sun','1','30'],index = df1.columns),
          ignore_index=True)

df1

In [42]:
# left_index right_index 按行索引进行合并
df1.merge(df2,
          left_index=True,
          right_index = True,
          how='outer')

,df1_name,Age,Weight,Name,Gender
0,San Zhang,23,45,San Zhang,F
1,Si Li,29,70,Si Li,M
2,Wu Wang,0,8,Liu Zhao,F
3,Qi Sun,1,30,NaN,NaN


In [44]:
# suffixes 的命名方式


df2['Weight']=['50','60','40']
df2

,Name,Gender,Weight
0,San Zhang,F,50
1,Si Li,M,60
2,Liu Zhao,F,40


In [47]:
# suffixes

df1.merge(df2,
         left_on='df1_name',
         right_on = 'Name',
         suffixes=['_after','_before'])

,df1_name,Age,Weight_after,Name,Gender,Weight_before
0,San Zhang,23,45,San Zhang,F,50
1,Si Li,29,70,Si Li,M,60


In [ ]:
# validate


#### 1.2. 索引连接


索引连接，就是以索引为键值，与值索引（列名）连接没有本质区别。

pandas中使用的是 **join函数**

**df.join**(other, on=None, how='left', lsuffix='', rsuffix='', sort=False)

参数说明：

|默认参数 | 使用说明|  
|---|---|  
|other| 另外的表 |
|on| 键值|
|how| 连接方式，默认 inner，其他可选{'left', 'right', 'outer'|
|lsuffix| 重复列的左缀 |
|rsuffix| 重复列的右缀 |

In [48]:
# 
df1 = pd.DataFrame({'Age':[20,30]},
                    index=pd.Series(
                    ['San Zhang','Si Li'],name='Name'))

df1

,Age
Name,
San Zhang,20
Si Li,30


In [52]:
# 
df2 = pd.DataFrame({'Gender':['F','M']},
                 index = pd.Series(['San Zhang','Wu Wang'],name = 'Name'))
df2

,Gender
Name,
San Zhang,F
Wu Wang,M


In [59]:
# how

df1.join(df2,how='outer')

,Age,Gender
Name,,
San Zhang,20.0,F
Si Li,30.0,NaN
Wu Wang,NaN,M


In [60]:
# lsuffix 左缀

df2['Age']=['12','13']
df2

,Gender,Age
Name,,
San Zhang,F,12
Wu Wang,M,13


In [65]:
# lsuffix 左缀
df1.join(df2,
         how = 'outer',
         lsuffix='_a',
         rsuffix='_b')

,Age_a,Gender,Age_b
Name,,,
San Zhang,20.0,F,12
Si Li,30.0,NaN,NaN
Wu Wang,NaN,M,13


### 二、方向连接

#### 2.1. concat

**concat**直接按照纵向或者横向拼接

**pd.concat**(objs,axis=0,join='outer',ignore_index: bool =False,keys=None,levels=None,names=None,verify_integrity: bool = False,sort: bool = False,copy: bool = True,)

参数说明

|默认参数 | 使用说明|  
|---|---|  
|objs| 合并对象的列表 |
|axis| 拼接的方向，axis=0，纵向拼接；axis=1，横向拼接 |
|join|  |
|ignore_index| 是否忽略原表索引  |
|keys|  |
|levels|  |
|names|  |



In [ ]:
#
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],
                 'Age':[20,30]})

df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})

In [66]:
# concat axis = 0
pd.concat([df1, df2])

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,NaN
San Zhang,12,F
Wu Wang,13,M


In [67]:
# concat axis =1

df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})

pd.concat([df1, df2, df3], 1)

,Age,Grade,Gender
0,NaN,80.0,M
1,NaN,90.0,F
San Zhang,20.0,NaN,NaN
Si Li,30.0,NaN,NaN


In [ ]:
# keys

#### 2.2. 序列与表的合并

把序列添加到表的行末或者列末，可以分别使用 **append** 和 **assign**

* **df.append**(other, ignore_index=False, verify_integrity=False, sort=False)

参数说明：

|默认参数 | 使用说明|  
|---|---|  
|other| 添加的序列 |
|ignore_index| 是否忽略索引 |
|verify_integrity|  bool, default False ,True 重复数据会报错|
|sort| 是否排序 |



* **df.assign**()

最好的添加列的方式是 df['new_col'] = ...



In [73]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],
                 'Age':[20,30]})

df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [72]:
# append
s = pd.Series(['Wu Wang', 21], index = df1.columns)
df1.append(s,ignore_index=True)


,Name,Age
0,San Zhang,20
1,Si Li,30
2,Wu Wang,21
